In [2]:
import cv2
import mediapipe as mp
import numpy as np
import winsound  # For beep sound on Windows

In [3]:
# Constants for EAR calculation
LEFT_EYE_INDEXES = [36, 37, 38, 39, 40, 41]
RIGHT_EYE_INDEXES = [42, 43, 44, 45, 46, 47]
EAR_THRESHOLD = 0.25
CONSECUTIVE_FRAMES = 30  # Number of frames to trigger alert

In [4]:
# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)

In [5]:
# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    A = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    B = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    C = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [6]:
# Function to play an alert sound
def play_alert_sound():
    frequency = 1000  # Set Frequency To 1000 Hertz
    duration = 1000   # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)

In [7]:
# Main function
def main():
    cap = cv2.VideoCapture(0)  # Use the default camera
    drowsiness_counter = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Get eye landmarks and scale them to the frame size
                h, w, _ = frame.shape
                left_eye = np.array([[face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h] for i in LEFT_EYE_INDEXES])
                right_eye = np.array([[face_landmarks.landmark[i].x * w, face_landmarks.landmark[i].y * h] for i in RIGHT_EYE_INDEXES])

                # Calculate EAR for both eyes
                left_ear = calculate_ear(left_eye)
                right_ear = calculate_ear(right_eye)
                avg_ear = (left_ear + right_ear) / 2.0

                # Drowsiness detection logic
                if avg_ear < EAR_THRESHOLD:
                    drowsiness_counter += 1
                    if drowsiness_counter >= CONSECUTIVE_FRAMES:
                        play_alert_sound()
                        cv2.putText(frame, "DROWSY!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
                else:
                    drowsiness_counter = 0

                # Draw eye landmarks
                for eye in [left_eye, right_eye]:
                    for (x, y) in eye:
                        cv2.circle(frame, (int(x), int(y)), 2, (0, 255, 0), -1)

        cv2.imshow("Drowsiness Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()